## 2.1
Write regular expressions for the following languages.
1. the set of all alphabetic strings;
2. the set of all lower case alphabetic strings ending in a $b$;
3. the set of all strings from the alphabet $a$,$b$ such that each $a$ is immediately preceded by and immediately followed by a $b$;

In [1]:
import re

regex1 = re.compile(r'[a-zA-Z]+')
regex2 = re.compile(r'[a-z]+b$')
regex3 = re.compile(r'b+(ab+)+')

## 2.2
Write regular expressions for the following languages. By “word”, we mean an alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, and so forth.
1. the set of all strings with two consecutive repeated words (e.g., “Humbert Humbert” and “the the” but not “the bug” or “the big bug”);
2. all strings that start at the beginning of the line with an integer and that end at the end of the line with a word;
3. all strings that have both the word grotto and the word raven in them (but not, e.g., words like grottos that merely contain the word grotto);
4. write a pattern that places the first word of an English sentence in a register. Deal with punctuation.

In [2]:
import re

regex1 = re.compile(r'\b(\w+) \1\b')
regex2 = re.compile(r'^\d.*\w+$')
regex3 = re.compile(r'\b(?=.*grotto)(?=.*raven).*\b')
regex4 = re.compile(r'^[^a-zA-z]*([a-zA-Z]+)')

## 2.3
Implement an ELIZA-like program, using substitutions such as those described on page 10. You might want to choose a different domain than a Rogerian psychologist, although keep in mind that you would need a domain in which your program can legitimately engage in a lot of simple repetition.

## 2.4
Compute the edit distance (using insertion cost 1, deletion cost 1, substitution cost 1) of “leda” to “deal”. Show your work (using the edit distance grid).

In [3]:
import pandas as pd

pd.DataFrame([[0, 1, 2, 3, 4],
              [1, 1, 2, 3, 3],
              [2, 2, 1, 2, 3],
              [3, 2, 2, 2, 3],
              [4, 3, 3, 2, 3]],
             index=tuple('#leda'), columns=tuple('#deal'))

,#,d,e,a,l
#,0,1,2,3,4
l,1,1,2,3,3
e,2,2,1,2,3
d,3,2,2,2,3
a,4,3,3,2,3


## 2.5
Figure out whether $drive$ is closer to $brief$ or to $divers$ and what the edit distance is to each. You may use any version of distance that you like.

In [4]:
import pandas as pd

pd.DataFrame([[0, 1, 2, 3, 4, 5, '-', 0, 1, 2, 3, 4, 5, 6, 7],
              [1, 1, 2, 3, 4, 5, '-', 1, 0, 1, 2, 3, 4, 5, 6],
              [2, 2, 1, 2, 3, 4, '-', 2, 1, 0, 1, 2, 3, 4, 5],
              [3, 3, 2, 1, 2, 3, '-', 3, 2, 1, 0, 1, 2, 3, 4],
              [4, 4, 3, 2, 2, 3, '-', 4, 3, 2, 1, 0, 1, 2, 3],
              [5, 5, 4, 3, 2, 3, '-', 5, 4, 3, 2, 1, 0, 1, 2]],
             index=tuple('#drive'), columns=tuple('#brief-#drivers'))

,#,b,r,i,e,f,-,#,d,r,i,v,e,r,s
#,0,1,2,3,4,5,-,0,1,2,3,4,5,6,7
d,1,1,2,3,4,5,-,1,0,1,2,3,4,5,6
r,2,2,1,2,3,4,-,2,1,0,1,2,3,4,5
i,3,3,2,1,2,3,-,3,2,1,0,1,2,3,4
v,4,4,3,2,2,3,-,4,3,2,1,0,1,2,3
e,5,5,4,3,2,3,-,5,4,3,2,1,0,1,2


## 2.6
Now implement a minimum edit distance algorithm and use your hand-computed results to check your code.

In [5]:
from typing import List, Union
import pandas as pd


def calc_min_edit_distance(source: str,
                           target: str,
                           del_cost: int = 1,
                           ins_cost: int = 1,
                           sub_cost: int = 1,
                           return_matrix: bool = False
                          ) -> Union[int, List[List[int]]]:
    n = len(source)
    m = len(target)
    d = [[0] * (m + 1) for _ in range(n + 1)]  # shape=(m+1, n+1)

    for i in range(1, n + 1):
        d[i][0] = d[i - 1][0] + del_cost
    for j in range(1, m + 1):
        d[0][j] = d[0][j - 1] + ins_cost

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            d[i][j] = min(d[i - 1][j] + del_cost,
                          d[i - 1][j - 1] + (0 if source[i - 1] == target[j - 1] else sub_cost),
                          d[i][j - 1] + ins_cost)

    if return_matrix:
        return d
    else:
        return d[-1][-1]


source = 'drive'
target = 'drivers'

d = calc_min_edit_distance(source, target)
print(f'minimum edit distance: {d}')

m = calc_min_edit_distance(source, target, return_matrix=True)
pd.DataFrame(m, index=tuple(f'#{source}'), columns=tuple(f'#{target}'))

minimum edit distance: 2


,#,d,r,i,v,e,r,s
#,0,1,2,3,4,5,6,7
d,1,0,1,2,3,4,5,6
r,2,1,0,1,2,3,4,5
i,3,2,1,0,1,2,3,4
v,4,3,2,1,0,1,2,3
e,5,4,3,2,1,0,1,2


## 2.7
Augment the minimum edit distance algorithm to output an alignment; you will need to store pointers and add a stage to compute the backtrace.

In [6]:
import random
from typing import List, Tuple
import pandas as pd


def calc_min_edit_distance(source: str,
                           target: str,
                           del_cost: int = 1,
                           ins_cost: int = 1,
                           sub_cost: int = 1,
                          ) -> Tuple[List[List[int]], List[List[List[Tuple[int, int]]]]]:

    n = len(source)
    m = len(target)
    d = [[0] * (m + 1) for _ in range(n + 1)]  # shape=(m+1, n+1)
    bp = [[[] for _ in range(m + 1)] for _ in range(n + 1)]  # shape=(m+1, n+1)

    for i in range(1, n + 1):
        d[i][0] = d[i - 1][0] + del_cost
        bp[i][0] = [(0, i - 1)]
    for j in range(1, m + 1):
        d[0][j] = d[0][j - 1] + ins_cost
        bp[0][j] = [(j - 1, 0)]

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            point2cost = {(j, i - 1): d[i - 1][j] + del_cost,
                          (j - 1, i - 1): d[i - 1][j - 1] + (0 if source[i - 1] == target[j - 1] else sub_cost),
                          (j - 1, i): d[i][j - 1] + ins_cost}
            min_cost = min(point2cost.values())
            d[i][j] = min_cost
            bp[i][j] = [p for p, c in point2cost.items() if c == min_cost]

    return d, bp


def backtrace(bp, seed=None):
    if seed is not None:
        random.seed(seed)

    path = [(len(bp[0]) - 1, len(bp) - 1)]

    id_x, id_y = -1, -1
    while True:
        id_xys = bp[id_y][id_x]
        if not id_xys:
            break
        id_x, id_y = random.choice(id_xys)
        path.append((id_x, id_y))

    return path[::-1]


source = 'intention'
target = 'execution'

d, bp = calc_min_edit_distance(source, target, sub_cost=2)
path = backtrace(bp, seed=42)
print('One of the paths:')
print(path)
pd.DataFrame(d, index=tuple(f'#{source}'), columns=tuple(f'#{target}'))

One of the paths:
[(0, 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (4, 4), (5, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]


,#,e,x,e,c,u,t,i,o,n
#,0,1,2,3,4,5,6,7,8,9
i,1,2,3,4,5,6,7,6,7,8
n,2,3,4,5,6,7,8,7,8,7
t,3,4,5,6,7,8,7,8,9,8
e,4,3,4,5,6,7,8,9,10,9
n,5,4,5,6,7,8,9,10,11,10
t,6,5,6,7,8,9,8,9,10,11
i,7,6,7,8,9,10,9,8,9,10
o,8,7,8,9,10,11,10,9,8,9
n,9,8,9,10,11,12,11,10,9,8
